In [14]:
from Crypto.PublicKey import RSA
import base64
from binascii import hexlify,unhexlify

In [15]:
# extended greater common divisor
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)


# modular inverse
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

    
# ciphertext to integer
def ct2int(ct):
    return int(hexlify(base64.b64decode(ct)), 16)


# integer to plaintext
def int2pt(m):
    hexm = hex(m)[2:]  # .strip("L")
    if len(hexm) % 2:
        hexm = '0' + hexm
    return unhexlify(hexm)

In [16]:
key1 = RSA.import_key(open("key1_pub.pem").read())
key2 = RSA.import_key(open("key2_pub.pem").read())

In [17]:
ct1 = "BzFd4riBUZdFuPCkB3LOh+5iyMImeQ/saFLVD+ca2L8VKSz0+wtTaL55RRpHBAQdl24Fb3XyVg2N9UDcx3slT+vZs7tr03W7oJZxVp3M0ihoCwer3xZNieem8WZQvQvyNP5s5gMT+K6pjB9hDFWWmHzsn7eOYxRJZTIDgxA4k2w="
ct2 = "jmVRiKyVPy1CHiYLl8fvpsDAhz8rDa/Ug87ZUXZ//rMBKfcJ5MqZnQbyTJZwSNASnQfgel3J/xJsjlnf8LoChzhgT28qSppjMfWtQvR6mar1GA0Ya1VRHkhggX1RUFA4uzL56X5voi0wZEpJITUXubbujDXHjlAfdLC7BvL/5+w="

In [18]:
c1 = ct2int(ct1)
c2 = ct2int(ct2)

In [19]:
c1

5050983197907648139720782448847677677343236446273586870502111273113384857588837608900494692102715861436825279596563904392832518247929761994240007673498974877828278590361242528762459283022987952424770766975922016521475963712698089809426428406068793291250622593222599407825968002220906973019105007856539702124

In [20]:
c2

99993713982446651581396992055360571139557381122865583938229634474666415937105325664345678113405954865343401854091338680448775405253508255042453184099961570780032181898606546389573694481401653361757628850127420072609555997892925890632116852740542002226555293049123266123721696951805937683483979653786235824108

In [22]:
n = key1.n 
e1, e2 = key1.e, key2.e
(_, s1, s2) = egcd(e1, e2)  # compute Bezout coefficients for e1 and e2

if s1 < 0:
    s1 = -s1
    c1 = modinv(c1, n)
elif s2 < 0:
    s2 = -s2
    c2 = modinv(c2, n)

m = (pow(c1,s1,n) * pow(c2,s2,n)) % n
print(f"msg int: {m}\nplaintext: {int2pt(m)}")

msg int: 62776018378826303731993505985858166542758896633026938340154202440070830295846836630500876932691035445174787046200753486570866078123449547598033197588810378299916556517155384672877661349185014100769763960938123717095837355876682453906268592788618322292439506868707727214682118601554445512691784759644852876334
plaintext: b'Yeah man, you got the message. The flag is W311D0n3! and this is a padding to have a long text, else it will be easy to decrypt.'


we need to compute $m = c_1^{s_1} \times c_2^{s_2} \mod n = (c_1^{s_1} \mod n) \times (c_2^{s_2} \mod n) \mod n$

but if $s_1 < 0$, to give meaning to this expression we have to compute the modular inverse of $c_1$ and raise that to the power of $|s_1| = -s_1$

same for $c_2$ and $s_2$...